In [1]:
from bs4 import BeautifulSoup
import requests
import Levenshtein

In [2]:
page = requests.get('https://icml.cc/Conferences/2021/AcceptedPapersInitial')
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
paper_list = []
author_and_place_list = []
for link in soup.find_all('p'):
    for paper in link.find_all('b'):
        paper_list.append(paper.get_text())
    for author_and_place in link.find_all('i'):
        author_and_place_list.append(author_and_place.get_text())
print(author_and_place_list)

['Mert Gurbuzbalaban (Rutgers University) · Umut Simsekli (Inria/ENS) · Lingjiong Zhu (Florida State University)', 'Alexander D Camuto (University of Oxford) · Xiaoyu Wang (Florida State University) · Lingjiong Zhu (Florida State University) · Christopher Holmes (University of Oxford) · Mert Gurbuzbalaban (Rutgers University) · Umut Simsekli (Inria/ENS)', 'Antoine Liutkus (Inria) · Ondřej Cífka (Télécom Paris, Institut Polytechnique de Paris) · Shih-Lun Wu (National Taiwan University) · Umut Simsekli (Inria/ENS) · Yi-Hsuan Yang (Academia Sinica) · Gaël RICHARD (Télécom Paris)', 'Martin Jørgensen (University of Oxford) · Søren Hauberg (Technical University of Denmark)', 'Haozhe Feng (State Key Lab of CAD&CG, Zhejiang University) · Zhaoyang You (Zhejiang University) · Minghao Chen (Zhejiang University) · Tianye Zhang (Zhejiang University) · Minfeng Zhu (State Key Lab of CAD&CG, Zhejiang University) · Fei Wu (Zhejiang University, China) · Chao Wu (Zhejiang University) · Wei Chen (State Ke

In [4]:
def find_ids(s, c):
    return [i for i in range(len(s)) if s[i] == c]

all_author = []
all_place = []
all_paper_info = [] # [paper_i] in which paper_i =[(author_i, place_i)]
for paper in author_and_place_list:
    paper_info = []
    group = paper.strip().split("\u00B7")
    for individual in group: 
        try:
            author, place = individual.split('(')
            author = author.strip()
            place = place.replace(')', '').replace('"', '').lower().strip()
            all_author.append(author)
            all_place.append(place)
        except:
#             print(individual)
#             pass
            items = individual.split('(')
            open_paren = find_ids(individual, '(')
            close_paren = find_ids(individual, ')')
            if open_paren[-1] > close_paren[0]:
                author = items[0] + items[1]
                place = items[2]
            else:
                author = items[0]
                place = items[1] + items[2]
            author = author.replace(')', '').strip()
            place = place.replace(')', '').replace('"', '').lower().strip()
            all_place.append(place)
        paper_info.append((author, place))
    all_paper_info.append(paper_info)

In [5]:
all_paper_info[1]

[('Alexander D Camuto', 'university of oxford'),
 ('Xiaoyu Wang', 'florida state university'),
 ('Lingjiong Zhu', 'florida state university'),
 ('Christopher Holmes', 'university of oxford'),
 ('Mert Gurbuzbalaban', 'rutgers university'),
 ('Umut Simsekli', 'inria/ens')]

In [6]:
from collections import Counter
import numpy as np
name, count = np.unique(all_place, return_counts=True)

In [7]:
print(len(name))

1094


In [8]:
for n in name:
    print (n)


-
.
1. state key laboratory for turbulence and complex systems, college of engineering, peking university; 2. institute of ocean research, peking university
aalto university
aalto university and university of manchester
aarhus university, madalgo
abacus.ai
academia sinica
accenture
adobe research
aer
agency for defense development
agency for science, technology and research a*star
ai innovation center, midea group
ai lab, samsung research china - beijing
ai research of jd.com
aignostics
aispeech
alan turing institute
alfréd rényi institute of mathematics
alibaba
alibaba damo academy
alibaba group
alibaba group u.s. inc.
alibaba group us
alibaba u.s. inc.
alibaba us
alibaba us, damo academy
allen institute for artificial intelligence
amazon
amazon adelaide
amazon ai
amazon alexa
amazon aws
amazon aws ai
amazon research
amazon research tuebingen
amazon search & ai
amazon web services
amazon.com
amazon.com, inc.
ambiata
american family insurance
amss,cas
aniti
ant financial
ant financial

mpi for intelligent systems tübingen, germany
mpi for intelligent systems, tübingen
mpi fuer informatik, saarbruecken
mpi-mis
mpi-sws
msr
msra
na
nagoya institute of technology
nagoya institute of technology / riken
nanjing university
nanjing university of science and technology
nankai university
nanyang technological univerisity
nanyang technological university
nanyang technology university, singapore
nanyang techonological university
national chiao tung university
national research council canada
national taiwan university
national technical university of athens
national tsing hua university
national univerisity of singapore
national university of defense technology
national university of singapore
national university of singapore; ihpc, singapore
naver ai lab
naver labs europe
nec / riken
nec biometrics research laboratories
nec/riken
netease fuxi ai lab
netflix
new york university
new york university shanghai
ninja theory
nlpr, casia
noah's ark lab
noah's ark lab, huawei technologi

In [9]:
alias_list = [
    ('aalto university | university of manchester', 'aalto university and university of manchester'),
    ('alibaba group', 'alibaba', 'alibaba damo academy', 'alibaba group', 'alibaba group u.s. inc.', 'alibaba group us', 'alibaba u.s. inc.', 'alibaba us', 'alibaba us, damo academy', 'damo academy, alibaba group,'),
    ('amazon', 'amazon adelaide', 'amazon ai', 'amazon alexa', 'amazon aws', 'amazon aws ai', 'amazon research', 'amazon research tuebingen', 'amazon search & ai', 'amazon web services', 'amazon.com', 'amazon.com, inc.'),
    ('ant financial', 'ant financial services group'),
    ('apple', 'apple ai/ml', 'apple inc.', 'apple, inc.'),
    ('arm research', 'arm'),
    ('army research lab', 'army research laboratory'),
    ('aurora innovation', 'aurora'),
    ('australian national university', 'australian national university, australia'),
    ('autodesk ai lab', 'autodesk'),
    ('aws', 'aws ai', 'aws ai labs', 'aws ai labs, amazon', 'aws berlin'),
    ('baidu', 'baidu inc.', 'baidu research', 'baidu research usa', 'baidu usa llc'),
    ('baidu | oregon state university', 'baidu research usa and oregon state university'),
    ('bar-ilan university', 'bar-ilan', 'bar ilan university'),
    ('bayer', 'bayer ag'),
    ('ben-gurion university', 'ben-gurion university of the negev'),
    ('benevolent ai', 'benevolentai'),
    ('university of virgina', 'biocomplexity institute and dept of computer science, university of virginia'),
    ('bosch', 'bosch center for artificial intelligence', 'bosch research center for ai, germany'),
    ('brown university', 'brown'),
    ('bytedance', 'bytedance ai lab', 'bytedance.inc'),
    ('california institute of technology', 'caltech'),
    ('california institute of technology | nvidia', 'caltech and nvidia'),
    ('carnegie mellon university', 'carnegie mellen university', 'cmu'),
    ('carnegie mellon university | bosch', 'carnegie mellon university / bosch center for ai'),
    ('chinese academy of sciences', 'chinese academy of science', 'institute of automation, chinese academy of sciences', 'institute of computing technology, cas', 'institute of computing technology, chinese academy of sciences', 'institute of information engineering, cas', 'institute of information engineering, cas, china', 'institute of information engineering, cas; school of cyber security, university of chinese academy of sciences', 'institute of information engineering, chinese academy of sciences', 'key laboratory of intelligent information processing, institute of computing technology, chinese academy of sciences', 'nlpr, casia', 'the state key laboratory for management and control of complex systems, institute of automation, chinese academy of sciences'),
    ('chung-ang university', 'chun-ang university', 'chung-ang univ., korea'),
    ('carnegie mellon university | facebook', 'cmu, fair'),
    ('carnegie mellon university | google brain', 'cmu, google brain'),
    ('columbia university', 'columbia'),
    ('university of illinois at urbana-campaign', 'computer science, university of illinois at chicago', 'illinois', 'university of illinois at chicago'),
    ('cornell university', 'cornell', 'cornell univeristy'),
    ('crest, ensae', 'crest/ensae'),
    ('criteo', 'criteo ai lab'),
    ('cns | university of paris', 'cnrs & irif, university of paris'),
    ('cnrs | criteo', 'cnrs and criteo ai lab'),
    ('cnrs | ens', 'cnrs and ens'),
    ('cnrs dauphine-psl | university of liverpool', 'cnrs dauphine-psl and university of liverpool'),
    ('cnrs | university of lille', 'cnrs, univ. lille'),
    ('cnrs | inria', 'cnrs/inria'),
    ('dartmouth college | facebook', 'dartmouth & facebook ai'),
    ('data61 | anu', 'data61 and anu'),
    ('deepmind | stanford university', 'deepmind & stanford university'),
    ('deepmind | inria | ens paris-saclay', 'deepmind / inria / ens paris-saclay'),
    ('deepmind | university of alberta', 'deepmind / univ alberta'),
    ('deepmind', 'deepmind technologies limited'),
    ('deepmind | jagiellonian university', 'deepmind, jagiellonian university'),
    ('deepmind | university college london', 'deepmind, ucl'),
    ('deepmind | university of alberta', 'deepmind, university of alberta'),
    ('deepmind | university of alberta', 'deepmind/university of alberta'),
    ('duke university', 'department of computer science, duke university', 'duke'),
    ('the hong kong polytechnic university', 'department of computing the hong kong polytechnic university', 'department of computing, the hong kong polytechnic university'),
    ('university of oxford', 'dept. of computer science, university of oxford'),
    ('ens | lighton', 'ecole normale supérieure & lighton'),
    ('ecole polytechnique', 'ecole polytechnique of paris', 'ecole polytechnique, paris, france', 'école polytechnique'),
    ('epfl', 'ecole polytechnique de lausann', 'epfl lausanne', 'epfl lausanne switzerland', 'école polytechnique fédérale de lausanne'),
    ('uc berkeley', 'eecs, uc berkeley'),
    ('ellis unit | university linz', 'ellis unit / university linz'),
    ('ens', 'ens lyon', 'ens paris', 'ens paris saclay', 'ens paris-saclay', 'ens rennes', 'ecole normale superieure', 'ecole normale supérieure', 'laboratoire de physique de l’ecole normale supérieure paris'),
    ('ens | facebook', 'ens / fair, paris'),
    ('ens | cnrs', 'ens and cnrs'),
    ('ens | inria', 'ens paris saclay & inria'),
    ('ensae | criteo', 'ensae & criteo ai lab'),
    ('ensai | crest', 'ensai, crest'),
    ('eth zurich', 'eth', 'eth zürich'),
    ('eth zurich | mpi-is', 'eth-z, mpi-is'),
    ('facebook', 'facebook ai', 'facebook ai applied research', 'facebook ai research fair', 'fair', 'facebook research', 'facebook ai research'),
    ('facebook', 'facebook inc', 'facebook inc, 1 hacker way, menlo park, ca 94025, united states', 'facebook inc.', 'facebook cds'),
    ('facebook | new york university', 'facebook / nyu'),
    ('facebook | university college london', 'facebook ai research & ucl'),
    ('facebook | university college london', 'facebook ai research & university college london'),
    ('facebook | tel aviv university', 'facebook ai research and tel aviv university'),
    ('facebook | paris dauphine university', 'facebook/dauphine'),
    ('facebook | uc berkeley', 'fair / uc berkeley'),
    ('fujitsu', 'fujitsu laboratories ltd.', 'fujitsu limited', 'fujitsu ltd.', 'fujitsu research america'),
    ('university college london', 'gatsby computational neuroscience unit', 'gatsby unit，ucl', 'ucl'),
    ('georgia tech', 'georgia institute of technology'),
    ('google', 'google ai', 'google research', 'google inc.', 'google israel', 'google research nyc', 'google research, ny', 'google, switzerland'),
    ('google brain', 'google brain robotics', 'google research, brain team'),
    ('deepmind', 'google deepmind'),
    ('google | university of michigan', 'google / u. michigan'),
    ('google | university of alberta', 'google / university of alberta'),
    ('google | tel aviv university', 'google and tel aviv university'),
    ('google brain | uc berkeley', 'google brain, uc berkeley'),
    ('google | university of colorado boulder', 'google research &            u. colorado boulder'),
    ('google | university of amsterdam', 'google research / university of amsterdam'),
    ('google | courant institute of mathematical sciences', 'google research and courant institute of mathematical sciences'),
    ('google | tel aviv university', 'google research and tel aviv university'),
    ('google | carnegie mellon university', 'google research, cmu'),
    ('google | harvard university', 'google/harvard'),
    ('google | university of chicago', 'google; university of chicago'),
    ('graviti technologies inc', 'graviti'),
    ('harbin institute of technology', 'harbin institute of technology shenzhen'),
    ('harvard university', 'harvard', 'harvard john a. paulson school of engineering and applied sciences', 'harvard medical school', 'harvard university, usa'),
    ('the hebrew university of jerusalem', 'hebrew university jerusalem israel', 'hebrew university of jerusalem', 'the hebrew university of jerusalem, il'),
    ('hec montreal | mila', 'hec montreal & mila'),
    ('heidelberg university | university of tuebingen', 'heidelberg university, university of tuebingen'),
    ('heidelberg institute for theoretical studies', 'hits - heidelberg institute for theoretical studies'),
    ('hong kong university of science and technology', 'hkust'),
    ('hong kong baptist university | riken', 'hkbu / riken'),
    ('huawei', 'huawei cloud & ai', 'huawei cloud ei', "huawei noah's ark lab", 'huawei tech. investment, co., ltd', 'huawei technologies', 'huawei technologies canada', 'huawei uk', "noah's ark lab", "noah's ark lab, huawei technologies", "noah's ark lab, huawei technologies.", "noah's ark laboratory, huawei", "noah's ark, huawei technologies"),
    ("huawei | higher school of economics", "huawei noah's ark lab; higher school of economics"),
    ('huazhong university of science and technology', 'huazhong university of science and technology, wuhan, china', 'hust'),
    ('ibm', 'ibm research', 'ibm research - tokyo', 'ibm research ai', 'ibm research, thomas j. watson research center', 'ibm thomas j. watson research center'),
    ('ideas ncbr | mim solutions | university of warsaw', 'ideas ncbr, mim solutions & university of warsaw'),
    ('columbia university', 'ieor department, columbia university'),
    ('university of illinois at urbana-campaign | google', 'illinois / google'),
    ('imperial college london', 'imperial college'),
    ('imperial college london | twitter', 'imperial college / twitter'),
    ('indian institute of science', 'indian institute of science iisc, bangalore', 'indian institute of science, bangalore, india'),
    ('indian institute of technology bombay', 'iit bombay'),
    ('indian institute of technology delhi', 'iit delhi'),
    ('indian institute of technology kanpur', 'iit kanpur'),
    ('indian institute of technology ropar', 'indian institute of technology, ropar'),
    ('indian institute of technology kharagpur', 'indian institute of technology, kharagpur'),
    ('indiana university | purdue university indianapolis', 'indiana   university-purdue university indianapolis'),
    ('indiana university', 'indiana university, bloomington'),
    ('inria', 'inria - up saclay', 'inria bordeaux - sud-ouest', 'inria lille - nord europe', 'inria paris', 'inria saclay', 'inria sophia antipolis', 'inria uca,'),
    ('inria | ens', 'inria - ecole normale superieure'),
    ('inria | ens', 'inria - ecole normale supérieure'),
    ('inria | ens | psl', 'inria - ecole normale supérieure - psl'),
    ('inria | university college london', 'inria, ucl'),
    ('inria | ens', 'inria, école normale supérieure'),
    ('inria | ens', 'inria/ens'),
    ('inria | google', 'inria/google'),
    ('inspir.ai', 'inspirai'),
    ('institute for infocomm research', 'institute for infocomm research,  singapore'),
    ('irisa | inria', 'irisa/inria'),
    ('isi foundation | boston university', 'isi foundation, boston university'),
    ('ist austria | neuralmagic', 'ist austria & neuralmagic'),
    ('istituto italiano di tecnologia | university college london', 'istituto italiano di tecnologia & university college london', 'istituto italiano di tecnologia and university college london'),
    ('jd digits', 'jd', 'jd ai research', 'jd.com'),
    ('johns hopkins university', 'johns hopkins university, usa'),
    ('kaist | aitrics', 'kaist, aitrics', 'kaist,aitrics'),
    ('kaust', 'king abdullah university  of science and technology', 'king abdullah university of science and technology', 'king abdullah university of science and technology.'),
    ('kc machine learning lab', 'kc ml2', 'kc-ml2'),
    ('kaust | mipt', 'kaust and mipt'),
    ('kaist', 'korea advanced institute of science and technology'),
    ('kth royal institute of technology', 'kth'),
    ('kth royal institute of technology | seb', 'kth royal institute of technology & seb'),
    ('kwai inc.', 'kwai seattle ai lab'),
    ('kwai inc. | university of rochester', 'kwai seattle ai lab, university of rochester'),
    ('kyoto university | riken', 'kyoto university / riken'),
    ('lawrence livermore national laboratory', 'lawrence livermore national laboratory llnl', 'lawrence livermore national laboroatory', 'lawrence livermore national labs', 'llnl'),
    ('leibniz university hannover', 'leibniz universität hannover'),
    ('lit ai lab', 'lit ai lab, institute for machine learning, johannes kepler university linz, austria'),
    ('lit ai lab | university linz', 'lit ai lab / university linz'),
    ('university of rouen normandie', 'litis lab, université de rouen normandie'),
    ('london school of economics', 'london school of economics and political science'),
    ('mit', 'massachusetts institute of technology', 'mit csail', 'mit lincoln laboratory', 'mit, mosaicml', ),
    ('max planck institute for intelligent systems | eth zurich', 'max planck institute for intelligent systems and eth zurich'),
    ('max planck institute for mathematics in the sciences | shanghai jiao tong university | university of new south wales', 'max planck institute for mathematics in the sciences; shanghai jiao tong university; university of new south wales'),
    ('mcgill university | mila', 'mcgill - mila'),
    ('mcgill university | deepmind', 'mcgill university / deepmind'),
    ('mcgill university | facebook', 'mcgill university / facebook'),
    ('mcgill university | mila', 'mcgill university / mila'),
    ('mcgill university | mila', 'mcgill university and mila'),
    ('mcgill university | facebook', 'mcgill, facebook'),
    ('mcgill university | mila', 'mcgill, mila'),
    ('mcgill university', 'mcgill'),
    ('memorial sloan kettering cancer center | vector institute | university of toronto', 'memorial sloan kettering cancer center, vector institute, and university of toronto'),
    ('microsoft', 'microsoft corporation', 'microsoft dynamics 365 ai', 'microsoft dynamics ai', 'microsoft research', 'microsoft research ai', 'microsoft research asia', 'microsoft research india', 'microsoft research montreal', 'microsoft research, new york'),
    ('midea group', 'ai innovation center, midea group'),
    ('mila', 'mila - quebec ai institute'),
    ('mila | mcgill university', 'mila / mcgill university', 'mila, mcgill university'),
    ('mila | university of montreal', 'mila university of montreal', 'mila, montreal', 'mila / university of montreal', 'mila, university of montreal', 'mila, université de montréal', 'mila / école polytechnique de montréal'),
    ('mila | university of montreal | samsung', 'mila, university of montreal & samsung sail montreal'),
    ('mit-ibm watson ai lab | ibm', 'mit-ibm watson ai lab, ibm research'),
    ('monash university', 'monash university, australia'),
    ('monash university | university of technology sydney', 'monash university / uts'),
    ('max planck institute for intelligent systems', 'mpi for intelligent systems', 'mpi for intelligent systems tübingen, germany', 'mpi for intelligent systems, tübingen'),
    ('max planck institute for informatics', 'mpi fuer informatik, saarbruecken'),
    ('max planck institute for mathematics in the sciences', 'mpi-mis'),
    ('max planck institute for software systems', 'mpi-sws'),
    ('nagoya institute of technology | riken', 'nagoya institute of technology / riken'),
    ('nanyang technology university', 'nanyang technology university, singapore', 'ntu'),
    ('national university of singapore', 'nus'),
    ('national university of singapore | institute of high performance computing', 'national university of singapore; ihpc, singapore'),
    ('naver', 'naver ai lab', 'naver labs europe'),
    ('nec | riken', 'nec / riken', 'nec/riken'),
    ('nec', 'nec biometrics research laboratories'),
    ('northeastern university', 'northeastern'),
    ('northwestern university', 'northwestern', 'northwestern u'),
    ('novoic', 'novoic ltd'),
    ('nippon telegraph and telephone', 'ntt', 'ntt corporation', 'ntt data mathematical systems inc.'),
    ('nvidia', 'nvidia corporation', 'nvidia research'),
    ('nvidia | bar-ilan university', 'nvidia / bar-ilan university'),
    ('nvidia | university of sydney', 'nvidia, university of sydney'),
    ('new york university', 'nyu'),
    ('new york university | uc berkeley', 'nyu/berkeley'),
    ('openai', 'open ai'),
    ('osaka university | riken', 'osaka university / riken aip'),
    ('otto-von-guericke university', 'otto von guericke universität magdeburg'),
    ('university of oxford', 'oxford'),
    ('university of oxford | deepmind', 'oxford and deepmind', 'oxford university & google deepmind'),
    ('peking university', 'peking university shenzhen graduate school'),
    ('penn state university', 'penn state', 'pennsylvania state university', 'the pennsylvania state university'),
    ('ping an technology', 'pingan technology'),
    ('university of montreal | puc-rio', 'polytechnique montreal & puc-rio'),
    ('university of montreal', 'polytechnique, montreal'),
    ('princeton university', 'princeton'),
    ("paris dauphine university | huawei", "psl - université paris dauphine & huawei noah's ark lab"),
    ('purdue university', 'purdue'),
    ('qualcomm', 'qualcomm ai research', 'qualcomm technologies, inc.'),
    ('radboud university | donders institute', 'radboud university / donders institute'),
    ('rensselaer polytechnic institute', 'renselaer polytechnic institute'),
    ('riken | the university of tokyo', 'riken / the university of tokyo'),
    ('riken', 'riken aip', 'riken center for brain science'),
    ('riken | kyoto university', 'riken aip / kyoto university'),
    ('rugters university', 'rutgers', 'rutgers university, new jersey'),
    ('salesforce', 'salesforce research'),
    ('samsung', 'samsung - sait ai lab, montreal', 'samsung advanced institute of technology', 'samsung research', 'samsung research china – beijing src-b', 'samsung research china, beijing', 'samsung research china-beijing src-b', 'samsung research institute china – beijing'),
    ('samsung | the university of edinburgh', 'samsung ai centre / university of edinburgh'),
    ('sap labs india', 'sap'),
    ('sapienza university of rome', 'sapienza university'),
    ('beijing institute of technology', 'school of automation, beijing institute of technology'),
    ('xidian university', 'school of electronic engineering, xidian university', "xi'dian university"),
    ('sun yat-sen university', 'school of intelligent systems engineering, sun yat-sen university'),
    ("xi'an jiaotong university", "school of software engineering, xi'an jiaotong university"),
    ('sensetime', 'sensetime group limited', 'sensetime group ltd.', 'sensetime ltd.', 'sensetime research'),
    ('sensetime | tetras.ai', 'sensetime&tetras.ai'),
    ('shanghai jiao tong university', 'shanghai jiao tong university of china', 'shanghai jiaotong university'),
    ('the chinese university of hong kong | chinese academy of sciences', 'shenzhen institute of artificial intelligence and robotics for society; the chinese university of hong kong, shenzhen shenzhen institutes of advanced technology, chinese academy of sciences'),
    ('sigopt | intel', 'sigopt/intel'),
    ('chinese academy of sciences | university of cas', 'sklois, institute of information engineering, chinese academy of sciences; scs, university of cas'),
    ('skoltech', 'skolkovo institute of science and technology'),
    ('singapore management university', 'smu_sg'),
    ('snap inc.', 'snap'),
    ('stanford university', 'stanford'),
    ('stanford university | databricks', 'stanford and databricks'),
    ('stanford university | apple', 'stanford university & apple'),
    ('stanford university | google', 'stanford university / google', 'stanford university, google research'),
    ('stanford university | vinai research', 'stanford university / vinai research'),
    ('state key lab of cad&cg | zhejiang university', 'state key lab of cad&cg, zhejiang university'),
    ('tau | google', 'tau, google'),
    ('technical university munich', 'technical university munich tum', 'tu munich', 'tum'),
    ('technion - israel institute of technology', 'technion', 'technion israeli institute of technology', 'technion- israel institute of technolgy, haifa', 'technion- israel institute of technology', 'the technion'),
    ('technical university of darmstadt', 'technische universität darmstadt'),
    ('technical university of berlin', 'technische universität berlin', 'tu berlin'),
    ('tel aviv university | google', 'tel aviv university and google', 'tel aviv university and google research', 'tel aviv university, google'),
    ('tel aviv university | facebook', 'tel-aviv university & facebook ai research'),
    ('tel aviv university', 'tel-aviv university'),
    ('tencent', 'tencent ai lab'),
    ('texas a&m university', 'tamu', 'texas a&m university, college station', 'texas a&m university-commerce'),
    ('the australian national university | data61 | csiro', 'the australian national university; data61, csiro'),
    ('the chinese university of hong kong', 'the chinese university of hongkong', 'the chinese university of hongkong, shenzhen'),
    ('university of texas at austin', 'the university of texas at austin', 'university of texas - austin', 'university of texas', 'university of texas, austin', 'ut austin', 'ut-austin'),
    ('the university of tokyo | riken', 'the university of tokyo / riken', 'the university of tokyo/riken'),
    ('the university of tokyo | riken | jst presto', 'the university of tokyo / riken / jst presto'),
    ('tsinghua university', 'tsinghua', 'tsinghua shenzhen international graduate school', 'tsinghua university, china'),
    ('toyota technological institute at chicago', 'ttic'),
    ('technical university of darmstadt', 'tu darmstadt'),
    ('technical university of dortmund', 'tu dortmund', 'tu dortmund university'),
    ('technical university of eindhoven', 'tu eindhoven'),
    ('technical university deflt', 'tu deflt'),
    ('technical university of kaiserslautern', 'tu kaiserslautern'),
    ('technical university of kaiserslautern | bosch', 'tu kaiserslautern/bosch center for artificial intelligence'),
    ('technical university of vienna', 'tu wien'),
    ('twitter | imperial college london', 'twitter, imperial college london'),
    ('telecom paris', 'télécom paris'),
    ('telecom paris | institut polytechnique de paris', 'télécom paris, institut polytechnique de paris'),
    ('university of toronto', 'u of toronto', 'univ. of toronto'),
    ('university of oxford', 'u oxford'),
    ('uc berkeley', 'u.c. berkeley', 'ucb', 'university of california at berkeley', 'university of california berkeley', 'university of california, berkeley', 'berkeley'),
    ('university of british columbia', 'ubc'),
    ('uber | university of toronto', 'uber atg / university of toronto'),
    ('uc berkeley | covariant', 'uc berkeley & covariant'),
    ('university college london', 'ucl'),
    ('uc los angeles', 'ucla', 'university of california los angeles', 'university of california at los angeles', 'university of california, los angeles'),
    ('uc los angeles | max planck institute for mathematics in the sciences', 'ucla math / stat; mpi mis'),
    ('uc santa barbara', 'ucsb', 'university of california at santa barbara', 'university of california, santa barbara'),
    ('uc santa cruz', 'ucsc', 'university of california santa cruz', 'university of california, santa cruz'),
    ('uc san diego', 'ucsd', 'university of california at san diego', 'university of california san diego', 'university of california, san diego'),
    ('umass amherst', 'umass', 'umass, amherst', 'university of massachusetts', 'university of massachusetts amherst', 'university of massachusetts, amherst'),
    ('university of maryland | university of north carolina at chapel hill', 'umd-cp & unc-ch'),
    ('university of north carolina at chapel hill', 'unc', 'unc chapel hill', 'university of north carolina chapel hill','unc-chapel hill'),
    ('university of michigan', 'univ michigan, ann arbor', 'university of michigan, ann arbor'),
    ('university grenoble alpes', 'univ. grenoble alpes'),
    ('university of southern california', 'univ. of southern california', 'usc'),
    ('university of washington', 'univ. of washington'),
    ('paris dauphine university', 'univ. paris dauphine', 'universite paris-dauphine', 'université paris-dauphine'),
    ('university carlos iii de madrid', 'universidad carlos iii de madrid'),
    ('university of chile', 'universidad de chile'),
    ('university of la república, uruguay', 'universidad de la república, uruguay'),
    ('university of pisa', 'universita di pisa'),
    ('university pompeu fabra', 'universitat pompeu fabra'),
    ('university of rouen normandie | criteo', 'universite de rouen normandie / criteo ai lab'),
    ('university of agder', 'university of agder, norway'),
    ('university of alberta | google brain', 'university of alberta / google brain'),
    ('university of alberta | alberta machine intelligence institute', 'university of alberta; alberta machine intelligence institute', 'university of alberta; amii'),
    ('university of amsterdam | qualcomm', 'university of amsterdam & qualcomm'),
    ('uc irvine', 'university of california, irivine', 'university of california, irvine'),
    ('uc riverside', 'university of california, riverside'),
    ('university of cambridge | max planck institute for intelligent systems', 'university of cambridge & mpi for intelligent systems, tübingen'),
    ('university of cambridge | uc los angeles', 'university of cambridge and ucla'),
    ('university of cambridge | alan turing institute', 'university of cambridge, alan turing institute'),
    ('university of chicago', 'university of chicago booth school of business'),
    ('university of colorado boulder', 'university of colorado'),
    ('university of freiburg | bosch', 'university of freiburg and bosch center for artificial intelligence'),
    ('university of genoa | istituto italiano di tecnologia', 'university of genoa & istituto italiano di tecnologia'),
    ('university of illinois at urbana-campaign', 'university of illinios at urbana-champaign', 'university of illinois', 'university of illinois, urbana champaign', 'uiuc'),
    ('university of illinois at urbana-campaign | mit', 'university of illinois at urbana-champaign/mit'),
    ('university of lille | inria lille', 'university of lille & inria lille'),
    ('university of maryland', 'university of maryland college park', 'university of maryland, college park'),
    ('university of milan | istituto italiano di tecnologia', 'university of milan & istituto italiano di tecnologia'),
    ('university of new south wales', 'university of new south wales, sydney'),
    ('university of oregon', 'university of oregon, usa'),
    ('university of pittsburgh | jd finance america corporation', 'university of pittsburgh & jd finance america corporation'),
    ('university of salzburg', 'university of salzburg, austria'),
    ('university of southern california', 'university of southern california sso', 'university of southern california usc'),
    ('university of texas at arlington | tencent', 'university of texas at arlington / tencent ai lab'),
    ('university of tokyo | riken', 'university of tokyo / riken'),
    ('university of toronto | vector institute', 'university of toronto and vector institute', 'university of toronto, vector institute'),
    ('university of toronto | nvidia', 'university of toronto, nvidia'),
    ('university of toronto | nvidia | vector institute', 'university of toronto, nvidia, vector institute', 'university of toronto, vector institute, nvidia'),
    ('university of toulouse iii', 'university of toulouse 3'),
    ('university of tuebingen', 'university of tübingen', 'universität tübingen'),
    ('university of tuebingen | international max planck research school for intelligent systems', 'university of tübingen, international max planck research school for intelligent systems'),
    ('university of warwick | alan turing institute', 'university of warwick & the alan turing institute'),
    ('university of washington | allen institue for ai', 'university of washington, allen institue for ai'),
    ('university of wisconsin-madison', 'university of wisconsin', 'university of wisconsin madison', 'university of wisconsin, madison', 'uw madison', 'uw-madison'),
    ('university of amsterdam', 'university van amsterdam'),
    ('university of potsdam', 'universität potsdam'),
    ("university côte d'azur", "université côte d'azur"),
    ('university of montreal', 'université de montréal'),
    ('university of montreal | mila', 'université de montréal; mila'),
    ('university of ottawa', 'université de ottawa'),
    ('university of picardie jules verne', 'université de picardie jules verne'),
    ('university of rouen normandie', 'université de rouen normandie', 'université de rouen normandie - litis'),
    ('laval university', 'université laval'),
    ('paris nanterre university', 'université paris nanterre'),
    ('paris dauphine university | riken', 'université paris-dauphine, psl research university/riken'),
    ('university of rennes 1 | inria', 'université rennes 1, irisa/inria rba'),
    ('toulouse capitole university | ens paris', 'université toulouse capitole & ens paris'),
    ('university of science and yechnology of china', 'ustc'),
    ('university of texas at austin | amazon', 'ut austin & amazon'),
    ('university of texas at austin | university of washington', 'ut-austin & university of washington'),
    ('university of tennessee', 'utk'),
    ('university of technology sydney', 'uts'),
    ('vector institute | university of toronto', 'vector institute & university of toronto'),
    ('university of warwick', 'warwick business school, the university of warwick'),
    ('washington university in st. louis', 'washington university in st louis'),
    ("xi'an jiaotong university | university of liverpool", "xi'an jiaotong-liverpool univ.", "xi'an jiaotong-liverpool university"),
    ('yale university', 'yale'),
    ('yonsei university', 'yonsei university, korea'),
    ('zhejiang university', 'zhejiang univ', 'zhejiang university, china'),
    ('george mason university', 'george mason univeristy')
]

In [10]:
len(alias_list)

347

In [11]:
def standardize(place): # return the standardized name for the place, using the alias list
    for group in alias_list:
        if place in group:
            return group[0]
    # if not found in alias list
    return place

all_paper_info = [[(author, standardize(place)) for author, place in paper_info] for paper_info in all_paper_info]        

In [12]:
all_paper_info[150]

[('Yanjun Han', 'stanford university'),
 ('Yining Wang', 'carnegie mellon university'),
 ('Xi Chen', 'new york university')]

In [13]:
all_paper_info_text = []

for paper_info in all_paper_info:
    paper_info_text = [author + '[' + place + ']' for author, place in paper_info]
    all_paper_info_text.append(' + '.join(paper_info_text))

In [14]:
all_paper_info_text[0]

'Mert Gurbuzbalaban[rutgers university] + Umut Simsekli[inria | ens] + Lingjiong Zhu[florida state university]'

In [15]:
all_paper_info_text[1]

'Alexander D Camuto[university of oxford] + Xiaoyu Wang[florida state university] + Lingjiong Zhu[florida state university] + Christopher Holmes[university of oxford] + Mert Gurbuzbalaban[rutgers university] + Umut Simsekli[inria | ens]'

In [16]:
with open('cleaned_affiliation_2021.txt', 'w+') as f:
    f.write('\n'.join(all_paper_info_text))